## Iterations 4

New concept: Instead of using Censu_OS version, we're going to split that into two columns Platform and OSBuild



In [29]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


In [30]:
file="50ksample.csv"

In [31]:
data_df=pd.read_csv(file)
data_df.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1514734,2b7b688e4a4315bc4f21f6849d03ab66,win8defender,1.1.15200.1,4.11.15063.0,1.275.828.0,0,7.0,0,NaN,...,33968.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
1,2926031,53f3fd511ba64acaec4e0c8559143301,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0,7.0,0,NaN,...,19951.0,0,0.0,0.0,0,0,0.0,1.0,5.0,0
2,7189617,ce48198690d1e4b437ffb60bfcb9e8df,win8defender,1.1.15100.1,4.18.1807.18075,1.273.516.0,0,7.0,0,NaN,...,33066.0,0,NaN,0.0,0,0,0.0,1.0,13.0,1
3,6264912,b3cfee470ee30c729d3a33f0017a8b51,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,...,12303.0,1,NaN,0.0,0,0,0.0,0.0,3.0,0
4,3321923,5f4f1fa2419439ff865d42a2a06ac8b5,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1486.0,0,7.0,0,NaN,...,4357.0,0,NaN,0.0,0,1,0.0,0.0,3.0,1


In [32]:
data_df=data_df[["EngineVersion","AVProductStatesIdentifier","AVProductsEnabled","Processor","SmartScreen","Census_MDC2FormFactor","Platform","OsBuild", "Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer","HasDetections"]]

data_df.head()


,EngineVersion,AVProductStatesIdentifier,AVProductsEnabled,Processor,SmartScreen,Census_MDC2FormFactor,Platform,OsBuild,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,1.1.15200.1,53447.0,1.0,x64,NaN,Desktop,windows10,15063,0.0,0.0,0
1,1.1.15200.1,53447.0,1.0,x64,RequireAdmin,Desktop,windows10,16299,0.0,1.0,0
2,1.1.15100.1,53447.0,1.0,x64,NaN,Notebook,windows10,15063,0.0,1.0,1
3,1.1.14600.4,47238.0,1.0,x64,NaN,Notebook,windows10,17134,0.0,0.0,0
4,1.1.15200.1,53447.0,1.0,x64,NaN,Desktop,windows10,17134,0.0,0.0,1


Drop that have nulls in AVProductStatesIdentifier

AVProductsEnabled fill nulls with 0
Browser Identtifier needs to be filled wiht 0
SmartScreen: clean up off and Off, on and On, “&#x02” and “&#x01" will have own columns., null should go to off bucket
Census_IsAlwaysConnectedCapable: replace with 0
Wdft_IsGamer: replace with 0



In [33]:
## Fix the nulls and add the substitutes
data_df[["AVProductsEnabled","Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer"]].fillna(value=0, inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["SmartScreen"].replace("Off", "off", inplace=True)
data_df["SmartScreen"].replace("On", "on", inplace=True)
data_df["SmartScreen"].fillna(value="off")
## This drops the nulls remaining in AVProductsStatesIdentifier
data_df.dropna(how='any',inplace=True)


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [34]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47734 entries, 0 to 49999
Data columns (total 11 columns):
EngineVersion                              47734 non-null object
AVProductStatesIdentifier                  47734 non-null float64
AVProductsEnabled                          47734 non-null float64
Processor                                  47734 non-null object
SmartScreen                                47734 non-null object
Census_MDC2FormFactor                      47734 non-null object
Platform                                   47734 non-null object
OsBuild                                    47734 non-null int64
Census_IsAlwaysOnAlwaysConnectedCapable    47734 non-null float64
Wdft_IsGamer                               47734 non-null float64
HasDetections                              47734 non-null int64
dtypes: float64(4), int64(2), object(5)
memory usage: 4.4+ MB


In [35]:
y=data_df["HasDetections"]
len(y)

47734

In [36]:
X=pd.get_dummies(data_df)
X=X.drop(["HasDetections"],axis=1)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,AVProductStatesIdentifier,AVProductsEnabled,OsBuild,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,EngineVersion_1.1.10302.0,EngineVersion_1.1.11701.0,EngineVersion_1.1.12101.0,EngineVersion_1.1.12603.0,EngineVersion_1.1.12805.0,...,Census_MDC2FormFactor_Detachable,Census_MDC2FormFactor_LargeTablet,Census_MDC2FormFactor_MediumServer,Census_MDC2FormFactor_Notebook,Census_MDC2FormFactor_PCOther,Census_MDC2FormFactor_SmallServer,Census_MDC2FormFactor_SmallTablet,Platform_windows10,Platform_windows7,Platform_windows8
0,53447.0,1.0,15063,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,53447.0,1.0,16299,0.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,53447.0,1.0,15063,0.0,1.0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,47238.0,1.0,17134,0.0,0.0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,53447.0,1.0,17134,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [37]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [38]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [39]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )



Training: 0.7109497206703911
Testing: 0.5990447461035696


In [40]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(0.4104960806664642, 'AVProductStatesIdentifier'),
 (0.1386466796991891, 'OsBuild'),
 (0.09817913342616202, 'SmartScreen_ExistsNotSet'),
 (0.033969118766172014, 'Wdft_IsGamer'),
 (0.03183246393955139, 'SmartScreen_RequireAdmin'),
 (0.030709743265411708, 'SmartScreen_off'),
 (0.02345222293676812, 'EngineVersion_1.1.15100.1'),
 (0.017388794364282166, 'AVProductsEnabled'),
 (0.015736473362504392, 'Census_MDC2FormFactor_Notebook'),
 (0.015235917053236682, 'EngineVersion_1.1.15200.1'),
 (0.013508964973558135, 'Census_MDC2FormFactor_Desktop'),
 (0.012698886714638747, 'Census_IsAlwaysOnAlwaysConnectedCapable'),
 (0.01025169122587966, 'EngineVersion_1.1.14901.4'),
 (0.009935759242133753, 'Census_MDC2FormFactor_Convertible'),
 (0.009789718854254274, 'EngineVersion_1.1.15000.2'),
 (0.009609033591770526, 'Processor_x86'),
 (0.009590605495631017, 'Processor_x64'),
 (0.008941155921527838, 'Census_MDC2FormFactor_AllInOne'),
 (0.0074700273791659146, 'EngineVersion_1.1.14800.3'),
 (0.0072373948616887

In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

             precision    recall  f1-score   support

          0       0.60      0.56      0.58      5912
          1       0.60      0.64      0.62      6022

avg / total       0.60      0.60      0.60     11934

